In [ ]:
#coding:UTF-8

import json
import re
import os
import requests
import sys
from bs4 import BeautifulSoup
def get_pages(newclass):
        '''获取该类新闻的总页数'''
        page = 1
        url = 'http://api.roll.news.sina.com.cn/zt_list?'
        params = {'channel':'news',
                'cat_1':newclass,
                'cat_2':'',
                'level':'',
                'show_ext':'1',
                'show_all':'1',
                'show_num':'1000',
                'tag':'1',
                'format':'json',
                'page':page
                }
        html = requests.get(url,params=params).content
        #print(bytes.decode(html))
        res=re.compile('"total":"(.*?)"')
        total = int(res.findall(bytes.decode(html))[0]) #获取该类新闻总条数

        #由每次请求1000条新闻，计算出总页数
        if total%1000 != 0:
            pages = (total/1000) + 1
        else:
            pages = total/1000
        return int(pages)

def get_new(newclass='gnxw',page=1):
        '''获取新闻标题与要闻'''
        new = []
        url = 'http://api.roll.news.sina.com.cn/zt_list?'
        params = {'channel':'news',
                'cat_1':newclass,
                'cat_2':'',
                'level':'',
                'show_ext':'1',
                'show_all':'1',
                'show_num':'1000',#每次请求1000个数据
                'tag':'1',
                'format':'json',
                'page':page
                }
        html = requests.get(url,params=params).content
        html = re.findall('\[\{(.*?)\}\]',bytes.decode(html))[0]
        html = html.split('},{')
        for s in html:
            s = '{' + s + '}'
            s = json.loads(s)

            #标题（title）与要闻用空格隔开保存到列表new里
            new.append(s['title'].replace('\n','')+' '+s['ext5'].replace('\n','')+'\n')
        return new

def Svae_new(page,path='D:/News/',filename='new.txt'):
    '''保存新闻到txt文档里面'''
    if not os.path.exists(path):
        os.mkdir(path)
    f = open(path+filename,'a')
    new = get_new(newclass,page)
    for i in new:
        f.write(i)
    f.close()
    print (str(page)+'/'+str(pages))



print( u'''
========================
    0：国内新闻
    1：国际新闻
    2：社会新闻
========================''')

news = ['gnxw','gjxw','shxw']
newclass = news[int(input(u'请输入数字获取对应新闻：'))]
pages = get_pages(newclass)-1

for page in range(1,pages+1):
    Svae_new(page,filename=newclass+'.txt')
